In [31]:
import numpy as np
import csv
from scipy import sparse
import pickle
import pandas as pd
import json

In [ ]:
pip install kaggle
export KAGGLE_USERNAME=eastonpotokar
export KAGGLE_KEY=1bfd140356aae8ffd62b234617295b77
kaggle datasets download -d shuyangli94/food-com-recipes-and-user-interactions

In [4]:
#save map from ingredients id to ingredients
maps = pickle.load( open('ingr_map.pkl', 'rb') )
ing_decoder = {i['id']: i['replaced'] for index, i in maps.iterrows()}
pickle.dump(ing_decoder, open("ingredient_decoder.pkl", "wb"))

In [45]:
#we put recipes into similar format as our dataset
recipes = pd.read_csv('PP_recipes.csv')

m = max(ing_decoder.keys()) + 1
n = len(recipes)

recipe_index = []
ingred_index = []
for index, recipe in recipes.iterrows():
    for i in json.loads(recipe['ingredient_ids']):
        recipe_index.append(index)
        ingred_index.append(i)
        
counts = [1]*len(recipe_index)

recipe_index += list(recipes.index)
ingred_index += [m-1]*len(list(recipes.index))
counts += list(recipes["calorie_level"])

X = sparse.csr_matrix((counts, [recipe_index, ingred_index]), shape=(n, m), dtype=np.float)

sparse.save_npz('recipes.npz', X)